In [ ]:
from datetime import datetime
import os

from astropy.time import Time
from bokeh.models import DatetimeTickFormatter, PrintfTickFormatter
from IPython.display import Markdown as md
import holoviews as hv
from holoviews import opts
hv.extension('bokeh', logo=False)
import numpy as np
import pandas as pd
import tables as tb

from lsst_efd_client import EfdClient

DATETIME_FORMAT = '%Y%m%d_%H%M%S'
REPORT_TIME_FORMAT = '%F %T'

In [ ]:
ipath = os.path.expanduser("~/Dropbox_LSST/Dome_Seeing_Monitor/DSM_Data/20200128")
date_str = "20200129_010709"
csc_index = 1

In [ ]:
ifname_centroid = os.path.join(ipath, f"smm_centroid_{date_str}.h5")

centroid_h5 = tb.open_file(ifname_centroid)
camera_info = centroid_h5.root.camera.info
general_info = centroid_h5.root.general.info

In [ ]:
md(f"### Dome Seeing Monitor Report {datetime.strptime(date_str, DATETIME_FORMAT).strftime(REPORT_TIME_FORMAT)}")

In [ ]:
ifname_psd = os.path.join(ipath, f"smm_psd_{date_str}.h5")
# Get data keys from file
psd_h5 = tb.open_file(ifname_psd)
glist = psd_h5.root._f_list_nodes(classname="Group")
keys = [k._v_name for k in glist]

In [ ]:
x_data = []
y_data = []
frequencies = None
datetimes = []
#print(keys)
for i, key in enumerate(keys):
    pd_h5 = pd.read_hdf(ifname_psd, key=key)
    x_data.append(pd_h5.X.values)
    y_data.append(pd_h5.Y.values)
    datetimes.append(datetime.strptime(key.replace('DT_', ''), DATETIME_FORMAT))
    if i == 0:
        frequencies = pd_h5.Frequencies.values

In [ ]:
print("General Information")
for name, value in zip(general_info.colnames, general_info[0]):
    print(f'{name}: {value.decode()}')
print()
print("Camera Information")
for name, value in zip(camera_info.colnames, camera_info[0]):
    try:
        v = value.decode()
    except AttributeError:
        v = value
    print(f'{name}: {v}')
print()
num_groups = len(list(centroid_h5.walk_nodes('/', 'Array'))) // 4
print(f"Number of data groups: {num_groups}")
duration = datetimes[-1] - datetimes[0]
print(f"Data Duration (H:M:S): {duration}")

In [ ]:
x = np.stack(x_data)
y = np.stack(y_data)
ts = np.array(datetimes, dtype=np.datetime64)

In [ ]:
psd_x = hv.Image((frequencies, ts, x), rtol=1, label='PSD X').opts(width=460)
psd_y = hv.Image((frequencies, ts, y), rtol=1, label='PSD Y').opts(yaxis='bare', width=350)

psd = psd_x + psd_y

colorbar_opts = {'colorbar': True, 'colorbar_position': 'bottom',
                 'colorbar_opts': {'formatter': PrintfTickFormatter(format="%0.0e")}}

dt_format = '%F %T'
formats = {'days': dt_format, 'months': dt_format, 'hours': dt_format, 'minutes': dt_format}
date_formatter = DatetimeTickFormatter(**formats)

psd.opts(opts.Image(cmap='viridis', logz=True, invert_yaxis=True,
                    **colorbar_opts,
                    height=400,
                    yformatter=date_formatter,
                    xlabel='𝜈 (Hz)', ylabel='Date'))

In [ ]:
client = EfdClient('summit_efd')

In [ ]:
start = Time(datetimes[0])
end = Time(datetimes[-1])
df = await client.select_time_series('lsst.sal.DSM.domeSeeing', '*', start.tai, end.tai, csc_index)
df_w = await client.select_time_series('lsst.sal.Environment.windSpeed', '*', start.tai, end.tai, csc_index)

In [ ]:
src = hv.Table(df.reset_index())
src_w = hv.Table(df_w.reset_index())
tick_rotation = 75
x_tuple = ('index', 'Date')
full_width = 350
end_col_size = full_width - 75

cx = hv.Curve(src, x_tuple, ('centroidX', 'X'), label='X')
cy = hv.Curve(src, x_tuple, ('centroidY', 'Y'), label='Y')
centroids = (cx * cy).opts(ylabel='Pixel Position', title='Centroids', width=full_width)
sx = hv.Curve(src, x_tuple, ('rmsX', 'X'), label='X')
sy = hv.Curve(src, x_tuple, ('rmsY', 'Y'), label='Y')
seeing = (sx * sy).opts(ylabel='scaled RMS (arcsec)', title='Seeing', width=full_width)
wind = hv.Curve(src_w, x_tuple, ('avg2M', 'Speed (m/s)'), label='Wind').opts(width=end_col_size)
flux = hv.Curve(src, x_tuple, ('flux', 'Flux'), label='Flux').opts(ylabel='')
max_adc = hv.Curve(src, x_tuple, ('maxADC', 'Max ADC'), label='Max ADC').opts(ylabel='')
fwhm = hv.Curve(src, x_tuple, ('fwhm', '(pixels)'), label='FWHM').opts(width=end_col_size)

layout = (centroids + seeing + wind + flux + max_adc + fwhm).cols(3)
layout.opts(opts.Layout(toolbar='above'),
            opts.Overlay(legend_position='right'),
            opts.Curve(xformatter=date_formatter, xrotation=tick_rotation, height=400))